### Resources
 
 'else' in a list comprehension<br>
 https://stackoverflow.com/questions/2951701/is-it-possible-to-use-else-in-a-list-comprehension
 
convert string to variable<br>
https://www.codeproject.com/Questions/1222606/Python-convert-string-and-variable-to-list-name

Apply function to each element of a list<br>
https://stackoverflow.com/questions/25082410/apply-function-to-each-element-of-a-list

In [1]:
# libraries, libraries!

import pandas as pd
import re # for RegEx
from re import search
from datetime import datetime

In [2]:
# get regex structure dataframes
structures = r'\\pre-cpt-fs\Profile-Documents$\PProfile Operations\Legal\Legal and Compliance Framework\Hilton\W\structures.xlsx'

# import the regex date formats
dfrgx = pd.read_excel(structures, sheet_name = 'dates')

# import the list of accrual holdings
accr = pd.read_excel(structures, sheet_name = 'accr', usecols=['accruals'])

# import the regex CLN formats
clnrgx = pd.read_excel(structures, sheet_name = 'cln')

# import the regex FRN formats
frnrgx = pd.read_excel(structures, sheet_name = 'frn')

# import South African holidays
sa_hols = pd.read_excel(structures, sheet_name = 'hols', usecols=['sa_hols'])

# import issuer regex formats
issrgx = pd.read_excel(structures, sheet_name = 'issuers')

# import CLNs
clns = pd.read_excel(r'\\fileserver.prescient.local\pim-business\Business Practice\Mcaps\CLNs.xlsx', sheet_name = 'CLN')

# import custodian accounts
sttlmnt = pd.read_excel(r'\\pre-cpt-fs\Profile-Documents$\PProfile Operations\Legal\Legal and Compliance Framework\Hilton\PFS Daily Compliance\2A - Fund Codes, Breach Register.xlsx', sheet_name = 'Sttlmnt')

# import CMS circular
med = pd.read_excel(r'\\fileserver.prescient.local\pim-business\Business Practice\Medical Schemes\20220117 MSA Circ6 Categorisations.xlsx', sheet_name = 'JSEEquity31Dec2021')

# import the Reg 28 file with all issuer codes assigned a named issuer
Reg28_basis = pd.read_excel(r'H:\Hilton\W\!Reg28Worx 30Jun2022.xlsx')

In [3]:
# utility function to open an excel file, .xls or .xlsx
def open_xl_file(file_name_and_path):
    import win32com.client as win32 # library to convert xls to xlsx
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    excel.DisplayAlerts = False # suppress the warning dialogue
    excel.Workbooks.Open(file_name_and_path)
    excel.DisplayAlerts = True # unsuppress the warning dialogue

In [4]:
# function to extract date from string

# define the date extractor function
def datex(txt):
    try:
        for pattern in dfrgx['date_regex']:
            if re.search(pattern, txt.title()):
                break
        return datetime.strptime(re.search(pattern, txt.title()).group(),dfrgx.loc[dfrgx['date_regex'] == pattern].iat[0,1]).strftime("%d%b%Y")

    # manage exceptions:
    except ValueError as ve:
        print(f'ValueError {ve}')
    except TypeError as te:
        print(f'TypeError {te}')
    except AttributeError as ae:
        print(f'AttributeError {ae}')
        
#test the datex function
txt = "BNP Paribas Personal Finance SA Ltd FRN BPPF31 Jb3+95 17-Aug-13"
datex(txt)

'17Aug2013'

In [5]:
# functions to extract CLN, FRN, NCD, ILB

def cln(txt):
    for pattern in clnrgx['regex']:
        if re.search(pattern, txt.upper()): 
            #return clnrgx.loc[clnrgx['regex'] == pattern].iat[0,2] # note 'break' below the first 'for'   
            return 1
        break
        
def frn(txt):
    for pattern in frnrgx['regex']:
        if re.search(pattern, txt.upper()): 
            #return frnrgx.loc[frnrgx['regex'] == pattern].iat[0,2] # note 'break' below the first 'for' 
            return 1
        break
        
# import the regex ILB formats
ilbrgx = pd.read_excel(structures, sheet_name = 'ilb')
def ilb(txt):
    for pattern in ilbrgx['regex']:
        if re.search(pattern, txt.upper()): 
            #return ilbrgx.loc[ilbrgx['regex'] == pattern].iat[0,2] # note 'break' below the first 'for'  
            return 1
        break       
        
txt = 'de Barge inflation asn'
print(frn(txt), cln(txt), ilb(txt))

None 1 1


In [6]:
# function that searches instrument description text for issuer

def issuer(txt):
    for i in range(len(issrgx)):
        for pattern in issrgx['regex']:
            if re.search(pattern, txt.upper()): 
                return issrgx.loc[issrgx['regex'] == pattern].iat[0,2] # note 'break' below the first 'for'   
        break
        
txt = 'The quick brown fox jumps over Standard Bank the lazy dog'
print(issuer(txt))

Standard Bank of South Africa Ltd


In [7]:
# test the issuer() FOR loop
txt = 'The quick brown fox jumped over the lazy dog'

for i in range(len(issrgx)):
    for pattern in issrgx['regex']:
        if re.search(pattern, txt.upper()):
            print(pattern, issrgx.loc[issrgx['regex'] == pattern].iat[0,2]) # note 'break' below the first 'for'           
    break

In [8]:
example = pd.read_excel('H:\Hilton\W\Reg 28 Report - Incl Effective Exposure.xlsx')
#example.iloc[:,1:5]

fl = r'H:\Hilton\W\Reg 28 Report - Incl Effective Exposure PD.xlsx'

# identify instrument attributes
example['Issuer'] = example['i Issue Name'].map(issuer)
example['CLN'] = example['i Issue Name'].map(cln)
example['FRN'] = example['i Issue Name'].map(frn)
example['ILB'] = example['i Issue Name'].map(ilb)
example['Date'] = example['i Issue Name'].map(datex)

# Map an if statement in Python
# https://stackoverflow.com/questions/29247718/map-an-if-statement-in-python
# example['Date'] = example['i Issue Name'].map(lambda x: )


example.to_excel(fl, index = False, sheet_name = 'R, b')

# open the file
open_xl_file(fl)

AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError 'NoneType' object has no attribute 'group'
AttributeError

In [9]:
# pandas map() Function – Examples
# https://sparkbyexamples.com/pandas/pandas-map-function-explained/
#   Conclusion: In this article, I have explained map() function
#   is from the Series which is used to substitute each value in a
#   series with another value and returns a Series object, since DataFrame
#   is a collection of Series, you can use the map() function to update the DataFrame.

# Syntax of Series.map()
# Series.map(arg, na_action=None)

In [10]:
# Create a pandas DataFrame.
import pandas as pd
import numpy as np
technologies= {
    'Fee' :[22000,25000,23000,np.NaN,26000],
    'Duration':['30days','50days','30days','35days','40days']
          }
df = pd.DataFrame(technologies)
print(df)

       Fee Duration
0  22000.0   30days
1  25000.0   50days
2  23000.0   30days
3      NaN   35days
4  26000.0   40days


In [11]:
# Using Lambda Function
df['Fee'] = df['Fee'].map(lambda x: x - (x*10/100))
print(df)

       Fee Duration
0  19800.0   30days
1  22500.0   50days
2  20700.0   30days
3      NaN   35days
4  23400.0   40days


In [12]:
# Using custom function
def fun1(x):
    return x/100
df['Fee'] = df['Fee'].map(lambda x:fun1(x))

In [13]:
# Let's add the currently to the Fee
df['Fee'] = df['Fee'].map('{} RS'.format)
print(df)

        Fee Duration
0  198.0 RS   30days
1  225.0 RS   50days
2  207.0 RS   30days
3    nan RS   35days
4  234.0 RS   40days


In [14]:
# Use na_action param
df['Fee'] = df['Fee'].map('{} RS'.format, na_action='ignore')
print(df)

           Fee Duration
0  198.0 RS RS   30days
1  225.0 RS RS   50days
2  207.0 RS RS   30days
3    nan RS RS   35days
4  234.0 RS RS   40days


In [15]:
# Using Dictionary for mapping
dict_map = {'30days':'35 Days','50days':'55 Days',
            '40days':'45 Days'}
updateSer = df['Duration'].map(dict_map)
df['Duration'] = updateSer
print(df)

           Fee Duration
0  198.0 RS RS  35 Days
1  225.0 RS RS  55 Days
2  207.0 RS RS  35 Days
3    nan RS RS      NaN
4  234.0 RS RS  45 Days


In [16]:
# Complete example of pandas map() function

# Create a pandas DataFrame.
import pandas as pd
import numpy as np
technologies= {
    'Fee' :[22000,25000,23000,np.NaN,26000],
    'Duration':['30days','50days','30days','35days','40days']
          }
df = pd.DataFrame(technologies)
print(df)

# Using Lambda Function
df['Fee'] = df['Fee'].map(lambda x: x - (x*10/100))
print(df)

# Using custom function
def fun1(x):
    return x/100
ser = df['Fee'].map(lambda x:fun1(x))
print(ser)

# Let's add the currently to the Fee
df['Fee'] = df['Fee'].map('{} RS'.format)
print(df)

df['Fee'] = df['Fee'].map('{} RS'.format, na_action='ignore')
print(df)

# Using Dictionary for mapping
dict_map = {'30days':'35 Days','50days':'55 Days',
            '40days':'45 Days'}
updateSer = df['Duration'].map(dict_map)
df['Duration'] = updateSer
print(df)

       Fee Duration
0  22000.0   30days
1  25000.0   50days
2  23000.0   30days
3      NaN   35days
4  26000.0   40days
       Fee Duration
0  19800.0   30days
1  22500.0   50days
2  20700.0   30days
3      NaN   35days
4  23400.0   40days
0    198.0
1    225.0
2    207.0
3      NaN
4    234.0
Name: Fee, dtype: float64
          Fee Duration
0  19800.0 RS   30days
1  22500.0 RS   50days
2  20700.0 RS   30days
3      nan RS   35days
4  23400.0 RS   40days
             Fee Duration
0  19800.0 RS RS   30days
1  22500.0 RS RS   50days
2  20700.0 RS RS   30days
3      nan RS RS   35days
4  23400.0 RS RS   40days
             Fee Duration
0  19800.0 RS RS  35 Days
1  22500.0 RS RS  55 Days
2  20700.0 RS RS  35 Days
3      nan RS RS      NaN
4  23400.0 RS RS  45 Days
